In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from src.config import CORRELATED_FEATURES_TO_REMOVE
from src.helpers import remove_correlations

In [4]:
from dotenv import load_dotenv

load_dotenv()

TRAIN_DATA_PATH = os.getenv("TRAIN_DATA_PATH")
TEST_DATA_PATH = os.getenv("TEST_DATA_PATH")


In [ ]:
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)

In [6]:
df_train.shape

(6244474, 11)

## **Preprocessing**

- DO NOT remove outliers (give signal)

In [ ]:
no_corr_train = remove_correlations(df_train, CORRELATED_FEATURES_TO_REMOVE)


(6244474, 8)